# pandas with Excel files and seaborn

In this session, you will be working with data contained in a spreadsheet. The file, named `Test_RR_TnTx_data.xlsx` contains three (3) separate sheets of Rainfall, Minimum Temperature and Maximum Temperature from the Ghana Meteorological Agency (GMet). The data is an *in situ* observation with a short term temporal span intended for the Python education. Kindly note that the acquired data must serve only the educational purpose intended.

We'll also be seeing how to use `seaborn` to produce useful plots for studying correlations and relationships in the observations. And we'll be using the `pymannkendall` package which implements the Mann-Kendall method for finding trends in the observations.

**Objectives** 

The following objectives are outlined for this lesson. Participants, at the end of the lesson should have adequate knowledge in the following:

1.	Loading data from an excel spreadsheet using Pandas, and some basic pandas features/ functionalities.
2.	Working with timeseries data
3.	Stacking Data using a defined function
4.	Data slicing, Groupby, etc
5.	Simple Computations and Data Description (sum, mean, percentiles)
6.	Moving Averages / Rolling Windows
7.	Quantifying trends
8.	Extreme Indices (Rnnmm, percentile differences)
9.	Visualization with seaborn



***Let's start by importing relevant packages.***

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb  # Useful for artistic and more scientific data visualization
import pymannkendall as mk  # Useful Python package for identifying Mann-Kendall trends in a dataset.
import calendar  # We need the calendar package to generate certain calendar properties
from datetime import datetime
from dateutil import parser
from pathlib import Path

<IPython.core.display.Javascript object>

In [3]:
data_path = Path("../data")

<IPython.core.display.Javascript object>

## 1. Loading data from `.xlsx` Excel spreadsheet file

We'll now read in the rainfall and temperature data from different sheets in a single `.xslx` Excel spreadsheet file.

Note: the `xlrd` package (which `pandas` used to use for reading `.xslx`-files [recently removed support for `.xslx`-files](https://stackoverflow.com/a/65266270/271776)). Fortunately `pandas` can use `openpyxl` instead, but unless you pandas version `>=1.2.0` you will have to tell `pandas` to use `openpyxl` using the `engine='openpyxl'` argument when opening a file.`

Sheet **RR** contains Daily Rainfall Totals, sheet **Tn** contains Daily Minimum Temperature and **Tx** contains the Daily Maximum Temperature data.

In [4]:
filepath_spreadsheet = data_path / "Test_RR_TnTx_data.xlsx"
# Rainfall
RR = pd.read_excel(filepath_spreadsheet, sheet_name="RR", engine="openpyxl")
# Minimum Temperature
TN = pd.read_excel(filepath_spreadsheet, sheet_name="Tn", engine="openpyxl")
# Maximum Temperature
TX = pd.read_excel(filepath_spreadsheet, sheet_name="Tx", engine="openpyxl")

<IPython.core.display.Javascript object>

In each sheet, the data is stored as:

- **Column A :** the Station Name (for 3 stations, namely: Tamale, Kumasi and Tema).
- **Column B :** abbreviation of parameter of interest (eg. RR for rainfall)
- **Column C :** Year of observation
- **Column D :** Month of observation
- **Columns E to AI :** Daily Data (Day 1 to maximum days in the month. Eg. Columns E to AI on the second rows contains data for 1st to 31st January 2010)

The first row contains the headers of each column and `pandas` uses this to set the column names

Let's have a look at the first few rows of the loaded rainfall data

In [5]:
RR.head()  

,Name,Eg El Abbreviation,Year,Month,Val01,Val02,Val03,Val04,Val05,Val06,...,Val22,Val23,Val24,Val25,Val26,Val27,Val28,Val29,Val30,Val31
0,Tamale,RR,2010,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Tamale,RR,2010,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,Tamale,RR,2010,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Tamale,RR,2010,4,0.0,0.0,0.0,0.0,0.9,0.0,...,89.1,0.0,0.0,0.0,73.3,1.5,0.0,0.0,0.0,NaN
4,Tamale,RR,2010,5,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.6


<IPython.core.display.Javascript object>

We set the day number as column headers.

In [6]:
RR.columns.values[4:] = np.arange(1, 32).astype("str")
TN.columns.values[4:] = np.arange(1, 32).astype("str")
TX.columns.values[4:] = np.arange(1, 32).astype("str")

<IPython.core.display.Javascript object>

In the next part, we select only the columns containing the year, month and daily data. We will be vertically stacking these using a defined function. In the end, the datasets should look somewhat like this:

19800101  RR1 <br>
19800102  RR2 <br>
... <br>
... <br>
... <br>
20201231 RRN

In [7]:
RR2s = RR.iloc[:, 2:]
TN2s = TN.iloc[:, 2:]
TX2s = TX.iloc[:, 2:]
RR2s.head(2)

,Year,Month,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,2010,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


<IPython.core.display.Javascript object>

---------------------------------------------------------------------------------------------------------------------------------
Wrangling the data
---------------------------------------------------------------------------------------------------------------------------------

**What is Data Wrangling?**

Data wrangling is the process of cleaning, structuring and enriching raw data into a desired format for better decision making in less time. Data wrangling is increasingly ubiquitous. In simpler terms, data wrangling (sometimes referred to as data munging) entails transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics. 
Data has become more diverse and unstructured, demanding increased time spent culling, cleaning, and organizing data ahead of broader analysis. <br>

**Reference:** <br>
> https://www.trifacta.com/data-wrangling/ <br>
> https://en.wikipedia.org/wiki/Data_wrangling <br><br><br>



With this premise established, we will wrangle the rainfall/temperature data, and this will be addressed as such.

> Vertically stacking the data using a DateTime index <br>
> Basic manipulation and dealing with missing values <br>
> Resampling to a different frequency
---------------------------------------------------------------------------------------------------------------------------------

In [8]:
# Let generate a new dataframe with index for each day within the observation period.
# We begin with the date creation using pandas date_range function

# Default frequency is daily
date = pd.date_range(start="1/1/2010", end="31/12/2013", freq="D")
type(date)  ### returns pandas.core.indexes.datetimes.DatetimeIndex

# Next, we populate a new dataframe with the actual data
NewRRData = pd.DataFrame(date, columns=["date"])
NewTNData = pd.DataFrame(date, columns=["date"])
NewTXData = pd.DataFrame(date, columns=["date"])

# Set the date as index of the new DataFrame.
NewRRData.set_index("date", inplace=True)
NewTNData.set_index("date", inplace=True)
NewTXData.set_index("date", inplace=True)

<IPython.core.display.Javascript object>

In [9]:
NewRRData.head()

""
date
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


<IPython.core.display.Javascript object>

Here, we will code a function that will help to transform the data in a column data timeseries. 

In [10]:
# Self-defined function to stack row-oriented data as columnar
def data_stacking(d, Data, s):
    year = np.int_(d["Year"])  # Format years as integers
    month = np.int_(d["Month"])  # Format months as integers

    # Let's create a blank column for the given station (s) which allows for appending data.
    Data[s] = None

    for i in d.index:
        # Looping through the years and month and producing the total number of days in each month of the year.
        # We employ calendar.monthrange() to acquire the total number of days in a given month.
        # Within the datasets, there are 31 column for each month data. By specifying the total number of actual days in a month, we omit the excess NaNs
        # in the end when matching data from the existing data set to the new vertically stacked data, using the dates as reference index.
        total_days = calendar.monthrange(
            datetime.strptime(str(year[i]), "%Y").year,
            datetime.strptime(str(month[i]), "%m").month,
        )[1]

        for day in range(
            1, total_days + 1
        ):  # for months with n days, we create a range of 1 to n+1 days to produce the exact n days. (eg. range(1,32) produces days 1 to 31)
            date_index = datetime.date(
                datetime.strptime(f"{year[i]} {month[i]} {day}", "%Y %m %d")
            )

            # Populating the new data with exact data from the old set for the same day.
            Data[s].loc[date_index] = d.iloc[i, day + 1]

    return Data  # return the stacked Data

<IPython.core.display.Javascript object>

#Start with one station and then move to multiples

In [11]:
# Selecting Data for only one station
d = RR2s.where(RR.Name == "Tema")

# Remove all the other rows where the year column contains NaNs. This will omit all other stations except Tema.
d = d[d["Year"].notna()]

<IPython.core.display.Javascript object>

In [15]:
d.shape, NewRRData.shape

((48, 33), (1461, 1))

<IPython.core.display.Javascript object>

In [12]:
# Single Station (Data Stacking via self-defined function)
NewRRData = data_stacking(d, NewRRData, "Tema")
NewRRData.head(2)

IndexError: index 96 is out of bounds for axis 0 with size 48

<IPython.core.display.Javascript object>

We attempt to loop through all the given stations and stack them to the new dataframe. To generate the list of stations without duplicates, we simply use the set() function which produce a set of any identified list without duplicates. We then pass it as a list to allow us loop through them easily, while stacking to the defined dataframe.

In [ ]:
#Creating Multiple columns for different stations
stations=sorted(list(set(RR.Name)))
for s in stations:
    d = RR2s.where(RR.Name==s)
    d = d[d['Year'].notna()]
    # for multi station data, we reset the index to start from 0 to the total rows per station data.
    d.index = range(len(d.index)) 
    NewRRData = data_stacking(d, NewRRData, s)

NewRRData.head(3)

Can we recreate the stacked data for the minimum and maximum temperature data too?
----------------------------------------------------------------------------------------------------------------------------

In [ ]:
stations=sorted(list(set(RR.Name)))
for s in stations:
    d=TN2s.where(TN.Name==s)
    d = d[d['Year'].notna()]
    # for multi station data, we reset the index to start from 0 to the total rows per station data.
    d.index=range(len(d.index)) 
    NewTNData=data_stacking(d, NewTNData, s)
    
    d=TX2s.where(TX.Name==s)
    d = d[d['Year'].notna()]
    # for multi station data, we reset the index to start from 0 to the total rows per station data.
    d.index=range(len(d.index)) 
    NewTXData=data_stacking(d, NewTXData, s)


In the next phase, we will perform data visualization with seaborn. <br >

Seaborn is a Python data visualization library based on matplotlib **(https://seaborn.pydata.org/)**. It provides a high-level interface for drawing attractive and informative statistical graphics.<br>

**Installation**<br>
Official releases of seaborn can be installed from PyPI using:

> **pip install seaborn**

Alternatively, you can install via the Anaconda distribution using:

> **conda install seaborn**



KDE Plot
----------------------------

Let's start off with data distribution plots using Seaborn's Kernel Density Estimates (KDE), which is a method for estimating the underlying distribution that data is sampled from by smoothing the sampled using a Gaussian kernel. KDE represents the data using a continuous probability density curve in one or more dimensions. There are other ways of estimating the distribution. <br>

Seaborn allows us to visualize the distribution of these observations, analagous to a histogram. 

In [ ]:
#### Sample Seaborn Visualizations
sb.kdeplot(NewRRData['Kumasi'].astype(float), legend=True, shade=True)   #Distribution plot
sb.kdeplot(NewRRData['Tema'].astype(float), legend=True, shade=True)   #Distribution plot

plt.legend(title='Stations', loc='upper right', labels=['Kumasi', 'Tema'])
plt.xlabel('Rainfall (mm)')

Scatter Plot
----------------------------
Scatter plots represent values for two different numeric variables. Scatter plots are used to observe relationships between the two variables.

> Let's attempt to see if there's a synergy in the rainfall events in any two locations (eg. Kumasi and Tema)

In [ ]:
sb.scatterplot(y='Kumasi', x='Tema', data=NewRRData)
plt.title("Rainfall in Kumasi against Tema")

Joint Plot
----------------------------
Seaborn's jointplot draws a plot of two variables with bivariate and univariate graphs. The bivariate plot (bottom left) produces a clustered diagram, which is useful in cluster analysis.

> Histograms rather produced an error when this command was originally issued. This may not be same in your case. However, as control, the dataset was formatted to floats by using the astype() method and passing the function float as an argument. 

In [ ]:
sb.jointplot(x='Tema', y='Kumasi', data=NewRRData.astype(float), kind='hex')
plt.suptitle("Rainfall in Kumasi against Tema \n\n\n")

Let's replicate the jointplot using monthly rainfall totals. Here, we will utilise the pandas groupby() function, in tandem with the Pandas Grouper class. <br>

Grouping Data with Groupby() function
----------------------------------------

The Pandas groupby() method allows for creating categories or groupings, for groupwise approximations, function applications, etc. <br>

eg. <br>
> Data.groupby(pd.Grouper(freq='Y')).aggregate(np.sum).plot()  #Annual Total <br>
> Data.groupby(pd.Grouper(freq='M')).aggregate(np.sum).plot()  #Monthly Total <br>

Alternatively, we can use the resample method (shown below): <br>

> Data=Data.astype(float).resample('M').sum()   #Monthly Total <br>
> Data=Data.astype(float).resample('Y').sum()   #Annual Total <br>

You notice that we utilise the pd.Grouper class. A Grouper allows the user to specify a groupby instruction for an object. This specification will select a column via the key parameter, or if the level and/or axis parameters are given, a level of the index of the target object. If axis and/or level are passed as keywords to both Grouper and groupby, the values passed to Grouper take precedence. In the example above, we pass the frequency as a datetime format (eg. "Y" for year, "M" for month, "MS" for start of month, etc.). This allows for grouping the dataframe into either yearly (first example) or monthly (second example) categories. <br>

Thereafter, we employ the aggregate function, while passing as argument, the numpy summation function (np.sum). This produces the summation of each monthly or yearly grouping.<br><br>

Okay......... Now, let's replicate the jointplot using monthly rainfall totals.


In [ ]:
sb.jointplot(x='Tema', y='Kumasi', data=NewRRData.groupby(pd.Grouper(freq='M')).sum().astype(float), kind='hex')
plt.suptitle("Rainfall in Kumasi against Tema \n\n\n")

Pair Plot
----------------------------
A simple way of identifying relations between each numerical series of a dataframe.

In [ ]:
sb.pairplot(NewRRData.groupby(pd.Grouper(freq='M')).sum().astype(float), diag_kind='hist')

Quick look into the Timeseries
----------------------------

In [ ]:
### Quicklook into the Data
cols_plot=stations
axes = NewRRData[cols_plot].plot(linestyle='-', figsize=(12, 8), subplots=True)
### NewRRData[cols_plot] aids in extracting the Rainfall data to visualize for each station.
for ax in axes:
    ax.set_ylabel('Rainfall (mm)')
    ax.set_ylim(0,150)

In [ ]:
### Now let's replicate for the daily minimum temperature data.
cols_plot=stations
axes = NewTNData[cols_plot].plot(linestyle='-', figsize=(12, 8), subplots=True)
for ax in axes:
    #ax.set_ylabel('Rainfall (mm)')
    ax.set_ylabel(r'Temp. ($^o$ C)')
    #ax.set_ylim(0,150)

Recap:  Grouping Data with Groupby() function
----------------------------------------

It allows for creating categories or groupings, for groupwise approximations, function applications, etc. <br>

eg. <br>
Data.groupby(pd.Grouper(freq='Y')).aggregate(np.sum).plot()  #Annual Total <br>
Data.groupby(pd.Grouper(freq='M')).aggregate(np.sum).plot()  #Monthly Total


In [ ]:
#### Moving Averages
Month_Data = NewRRData.groupby(pd.Grouper(freq='M')).aggregate(np.sum)
Month_Data.plot()
Month_Data.rolling(12).mean().plot()


Extreme Rainfall Events
-------------------------------------------------------------------------------------------------------------------------------
- **Rainy Days :** rainfall amount above 1 mm<br />
- **Rnnmm (eg. R10mm, R20mm, R50mm) :** events with rainfall amount above 10 mm, 20mm, and 50 mm respectively. <br>
- **Percentile** 

An alternative for identifying elements of a dataframe that meet a specific criteria is to pass the criteria within a square brackets attached to the dataframe. <br>

**For example:** <br>
> **a[a>1]** implies extracting all **a** elements where the value of **a** exceeds 1.<br><br>

We will employ this simplistic indexing method to map out rainy days and rainfall extremes.

In [ ]:
Rainy_days = Data[Data>=1].groupby(pd.Grouper(freq='M')).count() #Count of Rainy Days
R10mm = Data[Data>=10].groupby(pd.Grouper(freq='M')).count() #Count of R10mm events
R20mm = Data[Data>=20].groupby(pd.Grouper(freq='M')).count() #Count of R20mm events
R50mm = Data[Data>=50].groupby(pd.Grouper(freq='M')).count() #Count of R50mm events

**Let's quantify the rainfall extremes using the percentile approach**<br>

> 1. First, we compute the 95th percentile of the rainfall data using **.quantile(.95)** method. This serves as the threshold for extreme events. 
> 2. Next, we subtract the 95th percentile rainfall amount from the entire data series.

> 3. Afterwards, we employ the .where() method to map out where there is a positive difference (indicating an extreme event), and then ... 
> 4. we use .count() to quantify the number of these extreme events.

In [ ]:
perc=NewRRData['Kumasi'].quantile(.95)   #This produces the 95th percentile value.
diff=(NewRRData['Kumasi']-perc)
Events_95 = diff.where(diff>0).count()
percentage=round((Events_95/diff.count())*100, 3) #Compute the percentage of extreme events in a defined station
diff.plot(label='n = '+str(Events_95)+'\nPercent = '+str(percentage)+'%')
plt.axhline(y=0, color='r')
plt.legend()
plt.title('Events above the Percentile')



**Trend Assessmet using PyMannKendall**
--------------------------------------
Pymannkendall (**https://pypi.org/project/pymannkendall/**) allows for trend assessment in a dataset/observation. The package details out numerous sub-packages that are useful for various forms of trend assessment. For simplicity, we will focus on the Mann-Kendall original test. Nonetheless, feel free to try out its numerous trend assessment types after the class. <br> <br><br>

**Extra Article:** <br>
https://www.researchgate.net/publication/334688255_pyMannKendall_a_python_package_for_non_parametric_Mann_Kendall_family_of_trend_tests <br><br><br>

Let's start by importing the PyMannKendall package using <br>
> **import *pymannkendall* as *mk***


In [ ]:
import pymannkendall as mk
?mk.original_test

In [ ]:
#Trends In Data With PyMannKendall
mk.original_test(NewRRData['Kumasi'].astype(float))

The output from the above cell provides <br>

**Output:**
>    trend: tells the trend (increasing, decreasing or no trend)<br>
>    h: True (if trend is present) or False (if trend is absence)<br>
>    p: p-value of the significance test (eg. P<0.05 implies data is significant at 95% confidence level)  <br>
>    z: normalized test statistics <br>
>    Tau: Kendall Tau <br>
>    s: Mann-Kendal's score <br>
>    var_s: Variance S <br>
>    slope: sen's slope <br>

Next, we attempt to produce the slopes for each given station's monthly rainfall totals

In [ ]:
for station in stations:
    print(station,mk.original_test(NewRRData[s].groupby(pd.Grouper(freq='M')).sum().dropna().astype(float)).slope)

______________________________________________________________________________________________________________________________

Tasks / Activity
-------------------------------------------------------------------------------------------------------------------------------
For the stacked data: <br><br>
    1. Compute Monthly and Annual Mean Temperature <br>
    2. Identify the trends and magnitude of slope in the computed means in (1).<br>
    3. Using seaborn jointplot, let's produce a cluster plot of temperature and rainfall in any of the other locations. <br>

In [ ]:
Tavg=(NewTNData+NewTXData)/2

In [ ]:
### (1)
#Groupby Approach
MT_avg=Tavg.astype(float).groupby(pd.Grouper(freq='M')).mean()
AT_avg=Tavg.astype(float).groupby(pd.Grouper(freq='Y')).mean()

#or
#Resample Approach
MT_avg=Tavg.astype(float).resample('M').mean()
AT_avg=Tavg.astype(float).resample('Y').mean()

### (2)
Mtrend=mk.original_test(MT_avg['Kumasi'].astype(float))
Atrend=mk.original_test(AT_avg['Kumasi'].astype(float))


In [ ]:
### (2)
x=MT_avg.apply(standardized_anomaly)
y=NewRRData.astype(float).resample('M').sum()

h=sb.jointplot(y['Kumasi'],x['Kumasi']), kind='hex')
h.set_axis_labels('Temperature', 'Rainfall', fontsize=16)

------------------------------------------------------------------------------------------------------------------------------
Tasks / Activity
-------------------------------------------------------------------------------------------------------------------------------

    1. Let's produce standardized anomaly plots (formula below) of Daily Mean Temperature and visualize it.
    - First, build a function of standardized anomaly. 
     (You can use the default function build in python or better still, make use of lambda functions.)
    - Pass Tavg through the function, and plot out.

    NB: Standardized Anomaly (z) = (Ti - mean(T))/standard_deviation (T)

In [ ]:
### Approach 1
def standardized_anomaly(x):
    return (x-np.nanmean(x))/np.nanstd(x)

standardized_anomaly(Tavg).plot()

In [ ]:
#Approach 2
standardized_anomaly = lambda x:(x-np.nanmean(x))/np.nanstd(x)
Tavg.apply(standardized_anomaly).plot()
plt.axhline(y=0, color='k')

______________________________________________________________________________________________________________________________
THE END!!!!
------------------------------------------------------------------------------------------------------------------------------
